In [ ]:
import json
import zipfile
import requests
import rasterio
import numpy as np
import urllib.request
import shutil, glob, os
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as pyplt
from matplotlib.image import imsave

%matplotlib inline

## utils

In [ ]:
def make_url(root: str = "https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1&",
             width: str = 'width=4096&',
             height: str = 'height=4096&',
             srs: str = 'srs=epsg:4326',
             layers: list = None,
             bbox: str = None,
            ) -> str:
    return root+layers+bbox+width+height+srs


def download_all_layers(root: str = "https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1&",
                        width: str = 'width=4096&',
                        height: str = 'height=4096&',
                        srs: str = 'srs=epsg:4326',
                        layers: list = None,
                        bbox: str = None,
                        site: str = None,
) -> None:
    #cycle through each layer to download
    for layer in layers:
        # construct a URL string 
        url = root+layer+bbox+width+height+srs
        #download into a filename constructed as "site_layer.jpg"
        urllib.request.urlretrieve(url, site+'_'+layer.split('=')[-1].split('&')[0]+'.jpg')


def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    """
    response = filename for input
    destination = filename for output
    """    
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                

def unzip_nwpu(f):
    """
    f = file to be unzipped
    """    
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall()

## download test data (sentinel 2)

In [ ]:
#Lake Poopo, Bolivia
site='poopo'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2018&']
bbox = 'bbox=-68.66848367000537,-19.687928531849003,-66.67924128546656,-17.8774477409051&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Lake Urmia, Iran
site='urmia'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=44.587725529095295,36.86436828406643,46.230181583782795,38.50682433875393&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Lake Mead/ Lake Mojave, USA
site='mead_mojave'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-115.42507235769445,34.94273489926993,-113.78261630300695,36.58519095395743&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Aral Sea, Kazahkstan
site='aral_sea'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=58.032853536637845,43.955292007325,61.317765646012845,47.2402041167&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Copais Lake, Greece
site='copais'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=23.214268551013436,38.36067354565393,23.350911007068124,38.4633270490719&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Ramganga Lake, India
site='ramganga'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=78.63716910452058,29.468167756293038,78.91045401662996,29.673474763128976&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Qinghai Lake, China
site='qinghai'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=99.56471967977474,35.70094613663666,101.20168257039974,37.34340219132416&'
download_all_layers(site=site, layers=layers, bbox=bbox)

#Salton Sea, USA
site='salton_sea'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-116.21928854749297,32.903320741965295,-115.40080710218047,33.724548769309045&'
download_all_layers(site=site, layers=layers, bbox=bbox)

# Lake Faguibine, Mali
site='faguibine'
width = 'width=4082&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-4.656089323623854,14.959306189862815,-3.0191264329988536,16.601762244550315&'
download_all_layers(site=site, width=width, layers=layers, bbox=bbox)

# Mono Lake, USA
site='mono'
width = 'width=4082&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-119.22357779576633,37.797192162208084,-118.81433707311008,38.20780617587996&'
download_all_layers(site=site, width=width, layers=layers, bbox=bbox)

# Walker Lake, USA
site='walker'
width = 'width=4082&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-118.82841344222616,38.58649115903216,-118.62379308089804,38.791798165868094&'
download_all_layers(site=site, width=width, layers=layers, bbox=bbox)

# Lake Balaton, Hungary
site='balaton'
width = 'width=4096&'
height = 'height=3084&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=17.138306471163226,46.438781725159295,18.231446119600726,47.260009752503045&'
download_all_layers(site=site, width=width, height=height, layers=layers, bbox=bbox)

# Lake Koroneia, Greece
site='koroneia'
width = 'width=4096&'
height = 'height=3084&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=23.34167375183668,40.56321556296156,23.614958663946055,40.7685225697975&'
download_all_layers(site=site, width=width, height=height, layers=layers, bbox=bbox)

# Lake Salda, Turkey
site='salda'
width = 'width=4082&'
height = 'height=4096&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=29.6337346568769,37.49763563391751,29.736044837540963,37.60028913733548&'
download_all_layers(site=site, width=width, height=height, layers=layers, bbox=bbox)

# Lake Burdur, Turkey
site='burdur'
width = 'width=4096&'
height = 'height=2312&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=30.008970333642345,37.6345355820376,30.373579098290783,37.83984258887354&'
download_all_layers(site=site, width=width, height=height, layers=layers, bbox=bbox)

# Lake Mendocino, USA
site='mendocino'
width = 'width=4082&'
height = 'height=4096&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-123.19613698824355,39.189371563341254,-123.14498189791152,39.24069831505024&'
download_all_layers(site=site, width=width, height=height, layers=layers, bbox=bbox)

# Elephant Butte Reservoir, USA
site='elephant_butte'
width = 'width=4082&'
height = 'height=4096&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
bbox = 'bbox=-107.23803910498334,33.14614806405013,-107.13572892431928,33.2488015674681&'
download_all_layers(site=site, width=width, height=height, layers=layers, bbox=bbox)

In [ ]:
data_pth = Path('s2cloudless_imagery') / Path('data')
data_pth.mkdir(parents=True, exist_ok=True)

# cycle through each jpg image in the current directory
try:
    for f in glob.glob('*.jpg'):
        #move to the new directory
        if f.endswith('s2cloudless.jpg'): #2016 imagery
            shutil.move(f, 's2cloudless_imagery'+os.sep+'data'+os.sep+f.replace('s2cloudless.jpg','s2cloudless-2016.jpg'))
        else: #2017 or 2018 imagery
            shutil.move(f, 's2cloudless_imagery'+os.sep+'data')
except:
    pass


files = sorted(glob.glob('s2cloudless_imagery'+os.sep+'data'+os.sep+'*.jpg'))

sites = ['aral_sea','balaton','burdur','copais','elephant_butte', 
         'faguibine','koroneia','mead_mojave','mendocino','mono',
         'poopo','qinghai','ramganga','salda','urmia','walker']

## download training data (NWPU-RESISC45)

In [ ]:
destination = 'NWPU_images.zip'
file_id = '14kkcuU6wd9UMvjaDrg3PNI-e_voCi8HL'

download_file_from_google_drive(file_id, destination)

unzip_nwpu(destination)

os.rename('images','nwpu_images')

subdirs = [x[0] for x in os.walk('nwpu_images')][1:]
to_delete = [s for s in subdirs if 'lake' not in s]

for k in to_delete:
    shutil.rmtree(k, ignore_errors=True)

os.rename('nwpu_images'+os.sep+'lake','nwpu_images'+os.sep+'data')    

## working with labels

In [ ]:
labels_root = Path('s2cloudless_labels')
all_labels_json =  labels_root / 'all_labels.json'
data = json.load(open(all_labels_json))

key_aral_2018 = 'aral_sea_s2cloudless-2018.jpg'

data = data[key_aral_2018]

x1 = data['regions']['0']['shape_attributes']['all_points_x']
y1 = data['regions']['0']['shape_attributes']['all_points_y']

x2 = data['regions']['1']['shape_attributes']['all_points_x']
y2 = data['regions']['1']['shape_attributes']['all_points_y']

x3 = data['regions']['2']['shape_attributes']['all_points_x']
y3 = data['regions']['2']['shape_attributes']['all_points_y']

x4 = data['regions']['3']['shape_attributes']['all_points_x']
y4 = data['regions']['3']['shape_attributes']['all_points_y']


with rasterio.open('s2cloudless_imagery'+os.sep+'data'+os.sep+key_aral_2018) as dataset:
    rgb = dataset.read().T

print(f"rgb.shape: {rgb.shape}")
pyplt.imshow(rgb)
# swap y and x
pyplt.plot(y1,x1,'r')
pyplt.plot(y2,x2,'k')
pyplt.plot(y3,x3,'b')
pyplt.plot(y4,x4,'m')

In [ ]:
# get the dimensions of the image
nx, ny, nz = np.shape(rgb)
print(f"{nx:}, {ny:}, {nz:}")

mask = np.zeros((ny,nx))

X = [[x1], [x2], [x3], [x4]]
Y = [[y1], [y2], [y3], [y4]]

for x,y in zip(Y,X):
    # the ImageDraw.Draw().polygon function we will use to create the mask
    # requires the x's and y's are interweaved, which is what the following
    # one-liner does    
    polygon = np.vstack((x,y)).reshape((-1,),order='F').tolist()

    # create a mask image of the right size and infill according to the polygon
    if nx>ny:
        x,y = y,x 
        img = Image.new('L', (ny, nx), 0)
    else:
        img = Image.new('L', (nx, ny), 0)

    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    # turn into a numpy array
    m = np.array(img)
    mask = mask + m
    
pyplt.imshow(mask)

imsave("mask.jpg", mask.astype('uint8'))

In [ ]:
with rasterio.open('mask.jpg') as dataset:
     mask = np.flipud(np.rot90(dataset.read().T))

plt.imshow(rgb)        
plt.imshow(mask, cmap='gray', alpha=0.75)


In [ ]:
labels_root = Path('nwpu_labels')
nwpu_json =  labels_root / 'nwpu_lakes_20samplesA.json'
data = json.load(open(nwpu_json))

key = 'lake_024.jpg'

data = data[key]

x1 = data['regions']['0']['shape_attributes']['all_points_x']
y1 = data['regions']['0']['shape_attributes']['all_points_y']


with rasterio.open('nwpu_images'+os.sep+'data'+os.sep+key) as dataset:
    rgb = dataset.read().T

#print(f"rgb.shape: {rgb.shape}")
pyplt.imshow(rgb)
# swap y and x
pyplt.plot(y1,x1,'r')
